<a href="https://colab.research.google.com/github/brianramos/bots/blob/master/EntropyTSPwithAdaptiveGrids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import plotly.graph_objects as go
import random
import math
import time
import statistics
import numpy as np

class City:
    def __init__(self, x, y, name=None):
        self.x = x
        self.y = y
        self.name = name if name else str(id(self))
        self.visited = False
        self.entropy = 1.0

def calculate_global_entropy(cities, unvisited_cities, grid_resolution=None):
    width = max(city.x for city in cities)
    height = max(city.y for city in cities)

    if grid_resolution is None:
        grid_resolution = int(max(width, height) / 10) + 1

    num_cells = grid_resolution
    grid = np.zeros((num_cells, num_cells))
    cell_width = width / num_cells
    cell_height = height / num_cells

    unvisited_x = np.array([city.x for city in unvisited_cities])
    unvisited_y = np.array([city.y for city in unvisited_cities])
    cell_x = (unvisited_x / cell_width).astype(int)
    cell_y = (unvisited_y / cell_height).astype(int)
    cell_x = np.clip(cell_x, 0, num_cells - 1)
    cell_y = np.clip(cell_y, 0, num_cells - 1)

    for x, y in zip(cell_x, cell_y):
        grid[x, y] += 1

    probabilities = grid[grid > 0] / len(unvisited_cities)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

def euclidean_distance(city1, city2):
    return math.sqrt((city1.x - city2.x)**2 + (city1.y - city2.y)**2)

def generate_tsp_problem(num_cities, width, height):
    cities = []
    for i in range(num_cities):
        cities.append(City(random.randint(0, width), random.randint(0, height), name=f"City {i}"))
    return cities

def path_length(path):
    length = 0
    for i in range(len(path) - 1):
        length += euclidean_distance(path[i], path[i+1])
    length += euclidean_distance(path[-1], path[0])
    return length

def nearest_neighbor(cities):
    num_cities = len(cities)
    path = []
    current_city = random.choice(cities)
    current_city.visited = True
    path.append(current_city)

    for _ in range(num_cities - 1):
        min_distance = float('inf')
        next_city = None
        for city in cities:
            if not city.visited:
                distance = euclidean_distance(current_city, city)
                if distance < min_distance:
                    min_distance = distance
                    next_city = city

        if next_city:
            next_city.visited = True
            path.append(next_city)
            current_city = next_city
    return path

def two_opt(cities):
  num_cities = len(cities)
  path = cities.copy()
  improved = True
  while improved:
    improved = False
    for i in range(1, num_cities - 2):
        for j in range(i + 1, num_cities):
            if j - i == 1: continue
            new_path = path[:i] + path[i:j][::-1] + path[j:]
            if path_length(new_path) < path_length(path):
                path = new_path
                improved = True
                break
        if improved:
            break
  return path

def calculate_density(city, cities, radius):
    count = 0
    for other_city in cities:
        if other_city != city and euclidean_distance(city, other_city) <= radius:
            count += 1
    return count

def adaptive_contract_bubble(city, stretch, bubble_strength=0.5, density_radius=20, cities=None):
    density = calculate_density(city, cities, density_radius)
    adjusted_bubble_strength = bubble_strength * (1 - density / len(cities))
    city.entropy = max(0, city.entropy - (0.1 + 0.05 * stretch) * adjusted_bubble_strength)

def entropic_distance(city1, city2, stretch, initial_distances, global_entropy):
    initial_distance = initial_distances[(city1, city2)]
    entropy_factor = 1 / (city1.entropy * city2.entropy + 1e-10)
    stretch_factor = 1 + 0.1 * stretch
    global_entropy_weight = 1 + (global_entropy * 0.2)
    return initial_distance * entropy_factor * stretch_factor / global_entropy_weight

def traveling_salesperson_entropic_adaptive(cities, bubble_strength=0.5, density_radius=20, grid_resolution=None):
    num_cities = len(cities)
    path = []
    initial_distances = {}
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            distance = euclidean_distance(cities[i], cities[j])
            initial_distances[(cities[i], cities[j])] = distance
            initial_distances[(cities[j], cities[i])] = distance

    current_city = random.choice(cities)
    current_city.visited = True
    path.append(current_city)
    total_stretch = 0
    unvisited_cities = cities[:]
    for _ in range(num_cities - 1):
        global_entropy = calculate_global_entropy(cities, unvisited_cities, grid_resolution)
        min_distance = float('inf')
        next_city = None
        for city in cities:
            if not city.visited:
                distance = entropic_distance(current_city, city, total_stretch, initial_distances, global_entropy)
                if distance < min_distance:
                    min_distance = distance
                    next_city = city
        if next_city:
            total_stretch += euclidean_distance(current_city, next_city)
            adaptive_contract_bubble(current_city, total_stretch, bubble_strength, density_radius, cities)
            adaptive_contract_bubble(next_city, total_stretch, bubble_strength, density_radius, cities)
            next_city.visited = True
            path.append(next_city)
            current_city = next_city
            unvisited_cities.remove(next_city)
    return path, path_length(path)

def reset_cities(cities):
    for city in cities:
        city.visited = False
        city.entropy = 1.0

def tune_entropic_path(cities, iterations, bubble_strengths, grid_resolutions=None):
    best_path = None
    min_length = float('inf')
    best_params = None

    for _ in range(iterations):
        reset_cities(cities)
        bubble_strength = random.choice(bubble_strengths)
        grid_resolution = random.choice(grid_resolutions) if grid_resolutions else None

        path, length = traveling_salesperson_entropic_adaptive(cities.copy(), bubble_strength=bubble_strength, grid_resolution=grid_resolution)
        if length < min_length:
            min_length = length
            best_path = path
            best_params = (bubble_strength, grid_resolution)

    return best_path, min_length, best_params


def run_gather_statistics_iteratively(initial_city_count, iteration_count, step_size, num_iterations_per_run):
    city_counts = []
    entropic_path_lengths = []
    nn_path_lengths = []
    two_opt_path_lengths = []
    entropic_runtimes = []
    nn_runtimes = []
    two_opt_runtimes = []

    for city_count in range(initial_city_count, initial_city_count + iteration_count * step_size, step_size):
        print(f"Running for {city_count} cities...")
        city_counts.append(city_count)

        entropic_lengths_current = []
        nn_lengths_current = []
        two_opt_lengths_current = []
        entropic_runtimes_current = []
        nn_runtimes_current = []
        two_opt_runtimes_current = []
        for _ in range(num_iterations_per_run):
          cities = generate_tsp_problem(city_count, 100, 100)

          start_time = time.time()
          best_path, min_length, _ = tune_entropic_path(cities.copy(), 9, [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], grid_resolutions=[5, 10, 15])
          end_time = time.time()
          entropic_lengths_current.append(min_length)
          entropic_runtimes_current.append(end_time - start_time)

          start_time = time.time()
          reset_cities(cities)
          nn_path = nearest_neighbor(cities.copy())
          end_time = time.time()
          nn_lengths_current.append(path_length(nn_path))
          nn_runtimes_current.append(end_time - start_time)

          start_time = time.time()
          reset_cities(cities)
          two_opt_path = two_opt(cities.copy())
          end_time = time.time()
          two_opt_lengths_current.append(path_length(two_opt_path))
          two_opt_runtimes_current.append(end_time - start_time)

        entropic_path_lengths.append(statistics.mean(entropic_lengths_current))
        nn_path_lengths.append(statistics.mean(nn_lengths_current))
        two_opt_path_lengths.append(statistics.mean(two_opt_lengths_current))
        entropic_runtimes.append(statistics.mean(entropic_runtimes_current))
        nn_runtimes.append(statistics.mean(nn_runtimes_current))
        two_opt_runtimes.append(statistics.mean(two_opt_runtimes_current))


    fig = go.Figure()
    fig.add_trace(go.Scatter(x=city_counts, y=entropic_path_lengths, mode='lines+markers', name='Entropic Path Length'))
    fig.add_trace(go.Scatter(x=city_counts, y=nn_path_lengths, mode='lines+markers', name='Nearest Neighbor Path Length'))
    fig.add_trace(go.Scatter(x=city_counts, y=two_opt_path_lengths, mode='lines+markers', name='2-opt Path Length'))
    fig.update_layout(title='Scaling of TSP Algorithms', xaxis_title='Number of Cities', yaxis_title='Path Length')
    fig.show()

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=city_counts, y=entropic_runtimes, mode='lines+markers', name='Entropic Runtime'))
    fig2.add_trace(go.Scatter(x=city_counts, y=nn_runtimes, mode='lines+markers', name='Nearest Neighbor Runtime'))
    fig2.add_trace(go.Scatter(x=city_counts, y=two_opt_runtimes, mode='lines+markers', name='2-opt Runtime'))
    fig2.update_layout(title='Runtime of TSP Algorithms', xaxis_title='Number of Cities', yaxis_title='Runtime(seconds)')
    fig2.show()


initial_city_count = 5
iteration_count = 25
step_size = 1
num_iterations_per_run = 100

run_gather_statistics_iteratively(initial_city_count, iteration_count, step_size, num_iterations_per_run)

Running for 5 cities...
Running for 6 cities...
Running for 7 cities...
Running for 8 cities...
Running for 9 cities...
Running for 10 cities...
Running for 11 cities...
Running for 12 cities...
Running for 13 cities...
Running for 14 cities...
Running for 15 cities...
Running for 16 cities...
Running for 17 cities...
Running for 18 cities...
Running for 19 cities...
Running for 20 cities...
Running for 21 cities...
Running for 22 cities...
Running for 23 cities...
Running for 24 cities...
Running for 25 cities...
Running for 26 cities...
Running for 27 cities...
Running for 28 cities...
Running for 29 cities...
Running for 30 cities...
